In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import DebertaTokenizer, DebertaForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig
from sklearn.metrics import f1_score
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1'
# 自定义 Dataset 类
class EntityMatchingDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.sentence_1 = dataframe['sentence_1'].tolist()
        self.sentence_2 = dataframe['sentence_2'].tolist()
        self.labels = dataframe['label'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text1 = self.sentence_1[idx]
        text2 = self.sentence_2[idx]
        label = self.labels[idx]
        
        # 将两个句子连接在一起并进行 tokenization
        encoding = self.tokenizer.encode_plus(
            text1,
            text2,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 读取 CSV 数据



In [2]:
import torch

class GradientHook:
    def __init__(self, model):
        self.gradients = {}
        self.hooks = []
        
        # 为每一层注册钩子
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):  # 可以修改为其他层类型
                hook = module.register_backward_hook(self._save_gradient(name))
                self.hooks.append(hook)

    def _save_gradient(self, layer_name):
        def hook(module, grad_input, grad_output):
            self.gradients[layer_name] = grad_output[0].detach().cpu()  # 获取每层的梯度
        return hook

    def remove_hooks(self):
        for hook in self.hooks:
            hook.remove()

    def get_gradients(self):
        return self.gradients


In [3]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.gradient_hook = None  # 用于存储梯度钩子

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        self.gradient_hook = GradientHook(self.model)  # 在评估时启用梯度钩子
        result = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)  # 执行评估
        
        gradients = self.gradient_hook.get_gradients()  # 获取梯度信息
        self.gradient_hook.remove_hooks()  # 评估完毕后移除钩子

        # 在返回结果中加入梯度信息
        result["gradients"] = gradients
        return result

In [4]:
train_df = pd.read_json('/home/yanmy/Transfer-ER/Unicorn-main/data/Abt-Buy-Fewshot/train.json')
valid_df = pd.read_json('/home/yanmy/Transfer-ER/Unicorn-main/data/Abt-Buy-Fewshot/valid.json')
test_df = pd.read_json('/home/yanmy/Transfer-ER/Unicorn-main/data/Abt-Buy-Fewshot/test.json')
train_df.columns = ['sentence_1', 'sentence_2', 'label']
valid_df.columns = ['sentence_1', 'sentence_2', 'label']
test_df.columns = ['sentence_1', 'sentence_2', 'label']
# 加载 DeBERTa-v3-base 的 tokenizer
# tokenizer = DebertaTokenizer.from_pretrained('/home/yanmy/model/deberta-v3-base')
tokenizer = AutoTokenizer.from_pretrained('/home/yanmy/model/deberta-v3-base', padding_side="right")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

# 定义数据集
train_dataset = EntityMatchingDataset(train_df, tokenizer)
valid_dataset = EntityMatchingDataset(valid_df, tokenizer)
test_dataset = EntityMatchingDataset(test_df, tokenizer)

# 加载 DeBERTa-v3-base 模型
model = AutoModelForSequenceClassification.from_pretrained('/home/yanmy/model/deberta-v3-base', num_labels=2)

# 配置 LoRA
lora_config = LoraConfig(
    r=8,  # LoRA rank
    lora_alpha=16,  # 通常 LoRA alpha 设置为 2 倍的 rank
    target_modules=["value_proj"],  # 微调分类层
    lora_dropout=0.1,
    bias="none"
)
model = get_peft_model(model, lora_config)

# 定义 f1_score 评估函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    f1 = f1_score(labels, predictions)
    return {"f1": f1}

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="none"
)

# 使用 Trainer 进行训练
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=valid_dataset,
#     # compute_metrics=compute_metrics  # 设置 f1_score 作为评估指标
# )

# # 开始训练
# trainer.train()

# # 在测试集上进行评估
# metrics = trainer.evaluate(test_dataset)
# print(metrics)  # 输出包含 f1_score 的评估结果



trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)
trainer.train()
# 进行评估
metrics = trainer.evaluate(test_dataset)
print(metrics)

/home/yanmy/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/yanmy/anaconda3/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model check

[2024-11-13 19:47:20,947] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/yanmy/anaconda3/compiler_compat/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/yanmy/anaconda3/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/yanmy/anaconda3/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/yanmy/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/yanmy/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensor

Epoch,Training Loss,Validation Loss
1,No log,No log
2,No log,No log
3,No log,No log


/home/yanmy/anaconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /home/yanmy/model/deberta-v3-base - will assume that the vocabulary was not modified.
  warnings.warn(
/home/yanmy/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/yanmy/anaconda3/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/yanmy/anaconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /home/yanmy/model/deberta-v3-base - will assume

{'eval_runtime': 2.9041, 'eval_samples_per_second': 659.751, 'eval_steps_per_second': 2.755, 'epoch': 3.0, 'gradients': {}}


In [7]:
def predict(text1, text2):
    # 获取模型所在设备
    device = model.device
    
    # 将文本转换为张量
    inputs = tokenizer(text1, text2, return_tensors="pt", truncation=True, padding=True)
    
    # 将输入数据移到相同的设备上
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    # 进行推理
    outputs = model(**inputs)
    logits = outputs.logits
    
    # 获取预测类别
    predicted_class = logits.argmax().item()
    
    return predicted_class  # 1: 同一实体, 0: 不同实体

predict_all = []
from tqdm.notebook import tqdm
for sentence_1, sentence_2, label in tqdm(test_df.values):
    prediction = predict(sentence_1, sentence_2)
    predict_all.append(prediction)
label_all = test_df['label'].tolist()
from sklearn.metrics import f1_score
f1_score(y_true=label_all, y_pred=predict_all)

  0%|          | 0/1916 [00:00<?, ?it/s]

0.0